In [ ]:
"""import sys
from pathlib import Path
cwd = Path.cwd()
if (cwd / 'src').exists():
    ROOT = cwd
else:
    ROOT = cwd.parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))
if str(ROOT / 'src') not in sys.path:
    sys.path.insert(0, str(ROOT / 'src'))

import importlib
importlib.invalidate_caches()

# Install pysindy if needed: `pip install pysindy`
import numpy as np
print("Add code to prepare X matrix and run SINDy")"""

In [8]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import pysindy as ps

# Go up one directory from notebooks/
ROOT = Path.cwd().parent

# Add project root to PYTHONPATH
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

print("PYTHONPATH updated with:", ROOT)


from src.ml.sindy_skeleton import sindy_fit


PYTHONPATH updated with: d:\OneDrive\Documents\My Docs\CMU\Fall '25\Model & Sims\Project\ODE-to-Circadian-Clocks


In [9]:
PROC = ROOT / "notebooks" / "data" / "processed"

train_df = pd.read_csv(PROC / "pca_train.csv")
val_df   = pd.read_csv(PROC / "pca_val.csv")
test_df  = pd.read_csv(PROC / "pca_test.csv")

pc_cols = [c for c in train_df.columns if c.startswith("PC")]
print("Using PCs:", pc_cols)


Using PCs: ['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9', 'PC10', 'PC11', 'PC12', 'PC13', 'PC14', 'PC15', 'PC16', 'PC17', 'PC18', 'PC19', 'PC20', 'PC21', 'PC22', 'PC23', 'PC24', 'PC25', 'PC26', 'PC27', 'PC28', 'PC29', 'PC30', 'PC31', 'PC32', 'PC33', 'PC34', 'PC35', 'PC36', 'PC37', 'PC38', 'PC39', 'PC40', 'PC41', 'PC42', 'PC43', 'PC44', 'PC45', 'PC46', 'PC47', 'PC48', 'PC49', 'PC50', 'PC51', 'PC52', 'PC53', 'PC54', 'PC55', 'PC56', 'PC57', 'PC58', 'PC59', 'PC60', 'PC61', 'PC62', 'PC63', 'PC64', 'PC65', 'PC66', 'PC67', 'PC68', 'PC69', 'PC70', 'PC71', 'PC72', 'PC73']


In [10]:
# Debugging

# Check smoothing effect
from scipy.signal import savgol_filter
if X.shape[0] >= 11:
    X_smooth = savgol_filter(X, window_length=11, polyorder=3, axis=0)
    print("NaNs CREATED by savgol_filter:", np.isnan(X_smooth).sum())
else:
    print("Smoothing skipped (trajectory too short)")

print(train_df[train_df.subject == subj].head(20))

df = pd.read_csv(PROC / "pca_train.csv")
print("Any NaNs in pca_train.csv PCs? →", df[pc_cols].isna().sum().sum())
print(df[pc_cols].head())


NaNs CREATED by savgol_filter: 0
           gsm subject condition  t_idx  \
0   GSM1168586  BB0012         R      1   
1   GSM1168587  BB0012         R      2   
2   GSM1168588  BB0012         R      3   
3   GSM1168589  BB0012         R      4   
4   GSM1168590  BB0012         R      5   
5   GSM1168591  BB0012         R      6   
6   GSM1168592  BB0012         R      7   
7   GSM1168593  BB0012         R      8   
8   GSM1168594  BB0012         R      9   
9   GSM1168595  BB0012         R     10   
10  GSM1168596  BB0012         R     11   
11  GSM1168597  BB0012         R     12   
12  GSM1168598  BB0012         R     13   
13  GSM1168599  BB0012         R     14   

                                                 file         PC1        PC2  \
0   d:\OneDrive\Documents\My Docs\CMU\Fall '25\Mod...   64.168170 -27.418664   
1   d:\OneDrive\Documents\My Docs\CMU\Fall '25\Mod...  -33.991239  45.176533   
2   d:\OneDrive\Documents\My Docs\CMU\Fall '25\Mod...   25.048800  -2.073032   
3

In [11]:
models = {}

for subj, df in train_df.groupby("subject"):
    print(f"\n=== Training SINDy for subject {subj} ===")

    X = df[pc_cols].values.astype(float)
    t_raw = df["t_idx"].values.astype(float)

    # Drop rows with NaN
    mask = ~np.isnan(X).any(axis=1)
    X = X[mask]
    t_raw = t_raw[mask]

    # Normalize to uniform time ∈ [0,1]
    t = np.linspace(0, 1, len(t_raw))

    # Debugging:
    print("\n=== SUBJECT", subj, "===")

    print("PC NaN counts per column:")
    print(np.isnan(X).sum(axis=0))

    print("Rows with any NaN:")
    print(np.where(np.isnan(X).any(axis=1))[0])

    print("First 5 rows of X with NaNs:")
    bad_rows = np.where(np.isnan(X).any(axis=1))[0]
    if len(bad_rows) > 0:
        print(X[bad_rows[:5]])

    # End debugging
    
    model = sindy_fit(X=X, t=t, threshold=0.01, smooth=True)
    print(model)
    model.print()

    models[subj] = model

print("\nFinished SINDy training for all subjects.")



=== Training SINDy for subject BB0012 ===

=== SUBJECT BB0012 ===
PC NaN counts per column:
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Rows with any NaN:
[]
First 5 rows of X with NaNs:


c:\Users\Sanchitha\anaconda3\envs\scanpy-env\lib\site-packages\pysindy\utils\axes.py:389: RuntimeWarning: invalid value encountered in power
  results = super().__array_ufunc__(ufunc, method, *args, **kwargs)


ValueError: Input X contains NaN.

In [ ]:
example_subj = list(train_df["subject"].unique())[0]
print("Simulating subject:", example_subj)

df = train_df[train_df.subject == example_subj]
X = df[pc_cols].values.astype(float)
mask = ~np.isnan(X).any(axis=1)
X = X[mask]

t = np.linspace(0, 1, len(X))

model = models[example_subj]

# initial state
x0 = X[0]

# simulation grid
t_sim = np.linspace(0, 1, 200)
X_sim = model.simulate(x0, t_sim)


ValueError: Values in t should be in strictly increasing order.

In [ ]:
example_subj = list(train_df["subject"].unique())[0]
print("Simulating subject:", example_subj)

df = train_df[train_df.subject == example_subj]
X = df[pc_cols].values.astype(float)
mask = ~np.isnan(X).any(axis=1)
X = X[mask]

t = np.linspace(0, 1, len(X))

model = models[example_subj]

# initial state
x0 = X[0]

# simulation grid
t_sim = np.linspace(0, 1, 200)
X_sim = model.simulate(x0, t_sim)


In [ ]:
from scipy.interpolate import interp1d
from sklearn.metrics import mean_squared_error
import math

val_rmse = {}

for subj, model in models.items():
    df_val = val_df[val_df.subject == subj]
    if df_val.empty:
        continue  # some subjects may not land in val split

    # True data
    Xv = df_val[pc_cols].values.astype(float)
    mask = ~np.isnan(Xv).any(axis=1)
    Xv = Xv[mask]
    tv = np.linspace(0, 1, len(Xv))

    # Simulate model
    x0 = Xv[0]
    t_sim = np.linspace(0, 1, 200)
    X_sim = model.simulate(x0, t_sim)

    f = interp1d(t_sim, X_sim[:,0], fill_value="extrapolate")
    pred = f(tv)

    rmse = math.sqrt(mean_squared_error(Xv[:,0], pred))
    val_rmse[subj] = rmse

val_rmse


NameError: name 'model' is not defined

In [ ]:
# Interpolate simulation into val time-grid
from scipy.interpolate import interp1d

sim_func = interp1d(t_sim, X_sim[:,0], fill_value="extrapolate")

val_pred = sim_func(val_df["t_idx"].values)
val_true = val_df["PC1"].values

rmse = np.sqrt(np.mean((val_pred - val_true)**2))
rmse
